# Saving HDF5 files to visualise in Paraview

- Saving static fields to HDF5 and export Xdmf
- Saving timesteps to HDF5 and export Xdmf

In [ ]:
import numpy as np
import conduction
from time import clock
from petsc4py import PETSc
from mpi4py import MPI
comm = MPI.COMM_WORLD

In [ ]:
minX, maxX = 0.0, 1.0
minY, maxY = 0.0, 1.0
minZ, maxZ = 0.0, 1.0
nx, ny, nz = 30, 30, 30
n = nx*ny*nz

mesh = conduction.ConductionND((minX, minY, minZ), (maxX, maxY, maxZ), (nx,ny,nz))

In [ ]:
k0 = np.ones(n)
k  = np.ones(n)
H  = np.zeros(n)
a  = np.ones(n)
c = np.zeros(n)

# update properties
mesh.update_properties(k, H)

# BCs
mesh.boundary_condition('maxZ', 298.0, flux=False)
mesh.boundary_condition('minZ', 1e3, flux=True)

In [ ]:
def hofmeister1999(k0, T, a, c=0.0):
    return k0*(298.0/T)**a + c*T**3

def nonlinear_conductivity(self, k0, tolerance):
    k = k0.copy()
    self.construct_matrix()

    error = np.array(10.0)
    i = 0
    t = clock()

    while (error > tolerance):
        k_last = self.diffusivity[:].copy()
        self.diffusivity[:] = k
        self.construct_matrix()

        T = self.solve()
        k = hofmeister1999(k0, T, a, c)

        err = np.absolute(k - k_last).max()
        comm.Allreduce([err, MPI.DOUBLE], [error, MPI.DOUBLE], op=MPI.MAX)
        i += 1

        if comm.rank == 0:
            print("{:2} iterations in {:.2f} seconds, residual = {:.2e}".format(i, clock()-t, float(error)))

In [ ]:
# solve
nonlinear_conductivity(mesh, k0, 1e-5)
T = mesh.temperature[:]
q = mesh.heatflux()

In [ ]:
# save to hdf5
prefix = "timestep2"

mesh.save_mesh_to_hdf5(prefix + '.h5')
mesh.save_field_to_hdf5(prefix + '.h5', T=T)
conduction.generateXdmf(prefix + '.h5')

In [ ]:
steps = 10
H5_list = []
for step in xrange(steps):
    Ti = T/(step + 1)
    qi = np.ones_like(q)/(step + 1)
    
    filename = prefix + '_{:02d}.h5'.format(step)
    mesh.save_mesh_to_hdf5(filename)
    mesh.save_field_to_hdf5(filename, T=Ti)
    mesh.save_vector_to_hdf5(filename, q=qi)
    H5_list.append(filename)

In [ ]:
conduction.generateTimeseriesXdmf(*H5_list)